<a href="https://colab.research.google.com/github/ido177/colab/blob/main/window_func/window_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
6.33MB [00:00, 95.4MB/s]


# Загрузка БД

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
sqlite3.sqlite_version

'3.36.0'

In [3]:
con = sqlite3.connect('db')

In [4]:
!gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR

Downloading...
From: https://drive.google.com/uc?id=1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 39.0MB/s]


In [5]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [6]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


head выводит 5 первых значений, tail - 5 последних


In [7]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

to_datetime меняет формат данных в дату
% помогает указать переменную

In [8]:
df.dtypes

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

возможные типы данных

In [9]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

Заливаем таблицу в БД
Название, выбор бд, наличие индекса, замена существующих

In [10]:
def select(sql):
  return pd.read_sql(sql,con)

функция селекта

# JOIN

In [46]:
users = pd.DataFrame({'id': [1,2,3],
                      'name': ['john', 'gleb', 'tyrion']})

In [47]:
users

,id,name
0,1,john
1,2,gleb
2,3,tyrion


In [48]:
items = pd.DataFrame({'user_id': [1,3],
                      'item_name': ['hleb', 'gold'],
                      'value': [5, 100]})

In [49]:
items

,user_id,item_name,value
0,1,hleb,5
1,3,gold,100


In [50]:
users.to_sql('users', con, index=False, if_exists='replace')
items.to_sql('items', con, index=False, if_exists='replace')

In [51]:
sql = '''
select * from users t
left join items i 
on t.id = i.user_id
'''

In [52]:
select(sql)

,id,name,user_id,item_name,value
0,1,john,1.0,hleb,5.0
1,2,gleb,NaN,None,NaN
2,3,tyrion,3.0,gold,100.0


Left join - все слева и то, что нашлось, справа
where i.item_name is not null - имитирует inner

In [53]:
sql = '''
select * from users t
inner join items i 
on t.id = i.user_id
'''

In [54]:
select(sql)

,id,name,user_id,item_name,value
0,1,john,1,hleb,5
1,3,tyrion,3,gold,100


inner join - только при полных совпадениях, без NaN
нет совпадения по id - скипаем строку

In [55]:
users = pd.DataFrame({'id': [1,2,3],
                      'name': ['john', 'gleb', 'tyrion'],
                      'victory': [2,10,1]
                      })

добавляем повтор по id

In [56]:
sql = '''
select * from users t
left join items i 
on t.id = i.user_id
'''

In [57]:
sql = '''
select * from users t
inner join items i 
on t.id = i.user_id
'''

In [58]:
select(sql)

,id,name,user_id,item_name,value
0,1,john,1,hleb,5
1,3,tyrion,3,gold,100


In [59]:
t_left = select(sql)

In [60]:
t_left["value"].sum()

105

In [61]:
select(sql)

,id,name,user_id,item_name,value
0,1,john,1,hleb,5
1,3,tyrion,3,gold,100


In [62]:
t_right = select(sql)

In [63]:
t_right['value'].sum()

105

In [66]:
sql = '''
select t.id, t.name, sum(t.value) as value_sum, count(i.item_name) as item_cnt, sum(i.value) as value_s

from users t
left join items i 
on t.id = i.user_id

group by t.id, t.name
'''

In [67]:
select(sql)

DatabaseError: ignored

In [68]:
sql = '''
select t.id, t.name, t.value as value_sum, count(i.item_name) as item_cnt, sum(i.value) as value_s

from users t
left join items i 
on t.id = i.user_id

group by t.id, t.name, t.value
'''

In [69]:
select(sql)

DatabaseError: ignored

In [ ]:
sql = ''' 
with items_agg as (
select t.user_id, count(t.item_name) as item_cnt, sum(value) as value_sum 
from items t
group by t.user_id
)

select *
from users t

left join items_agg i 
on t.id = i.user_id'''

In [ ]:
select(sql)

In [ ]:
sql = '''
select * from users t
FULL join items i 
on t.id = i.user_id
'''

FULL JOIN - возвращает все

In [ ]:
sql = '''
select * from users t
left join items i 
on t.id = i.user_id
union
select * from items i
left join users t
on i.user_id = t.id
'''

эквивалент full join, но с left

In [ ]:
select(sql)

In [ ]:
sql = ''' select * from date
join month on 1 = 1

cross join 1 = 1. Селектит бещз привязки к колонкам

In [70]:
t = pd.DataFrame({'dt':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03'], format='%Y-%m-%d'),
                                       'revenue': [1,2,3]})

In [ ]:
t

In [71]:
t.to_sql('revenue', con, index=False, if_exists='replace')

In [ ]:
sql = ''' 
select * from revenue t
join revenue y 
on t.dt >= y.dt
'''

In [ ]:
select(sql)

In [ ]:
sql = ''' 
select t.dt, t.revenue, sum(y.revenue) as cum_sum from revenue t
join revenue y 
on t.dt >= y.dt
group by t.dt, t.revenue
'''

делаем нарастающий итог

In [ ]:
select(sql)

# window func

In [84]:
t = pd.DataFrame({'user_id':[1,1,1,2,2,2],
                  'dt':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03',
                                       '2021-04-01','2021-04-02','2021-04-03'], format='%Y-%m-%d'),
                  'revenue': [1,2,3,2,3,4]})

t.to_sql('revenue', con, index=False, if_exists='replace')

In [85]:
t

,user_id,dt,revenue
0,1,2021-04-01,1
1,1,2021-04-02,2
2,1,2021-04-03,3
3,2,2021-04-01,2
4,2,2021-04-02,3
5,2,2021-04-03,4


In [86]:
sql = '''select t.*,
sum(t.revenue) over (order by t.dt) as cum_sum
from revenue t'''

In [87]:
select(sql)

,user_id,dt,revenue,cum_sum
0,1,2021-04-01 00:00:00,1,3
1,2,2021-04-01 00:00:00,2,3
2,1,2021-04-02 00:00:00,2,8
3,2,2021-04-02 00:00:00,3,8
4,1,2021-04-03 00:00:00,3,15
5,2,2021-04-03 00:00:00,4,15


In [88]:
sql = '''select t.*,
sum(t.revenue) over (partition by t.user_id order by t.dt) as cum_sum
from revenue t'''

Партиция это окно, мы делаем окно для каждого пользователя, чтобы считать его отдельно, а не всех сразу(за определенный промежуток времени в примере)

In [89]:
select(sql)

,user_id,dt,revenue,cum_sum
0,1,2021-04-01 00:00:00,1,1
1,1,2021-04-02 00:00:00,2,3
2,1,2021-04-03 00:00:00,3,6
3,2,2021-04-01 00:00:00,2,2
4,2,2021-04-02 00:00:00,3,5
5,2,2021-04-03 00:00:00,4,9


In [134]:
t = pd.DataFrame({'user_id':[1,1,1,1,2,2,2],
                  'dt':pd.to_datetime(['2021-04-01','2021-04-02','2021-04-03','2021-04-03',
                                       '2021-04-03','2021-04-04','2021-04-05'], format='%Y-%m-%d'),
                  'revenue': [1,2,0,0,2,3,4]})

t.to_sql('revenue', con, index=False, if_exists='replace')

In [135]:
t

,user_id,dt,revenue
0,1,2021-04-01,1
1,1,2021-04-02,2
2,1,2021-04-03,0
3,1,2021-04-03,0
4,2,2021-04-03,2
5,2,2021-04-04,3
6,2,2021-04-05,4


In [136]:
sql = '''
select t.*, 
rank() over(partition by t.user_id order by t.dt desc) as rnk
from revenue t
'''

rank() ранжирует партицию, пишет его номер после сортировки

In [137]:
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,0,1
1,1,2021-04-03 00:00:00,0,1
2,1,2021-04-02 00:00:00,2,3
3,1,2021-04-01 00:00:00,1,4
4,2,2021-04-05 00:00:00,4,1
5,2,2021-04-04 00:00:00,3,2
6,2,2021-04-03 00:00:00,2,3


In [128]:
sql = '''
select t.*, 
dense_rank() over(partition by t.user_id order by t.dt desc) as rnk
from revenue t
'''

то же самое, что просто rank(), но не пропускает значения при повторении строк(смотри rnk = 2)

In [129]:
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,3,1
1,1,2021-04-03 00:00:00,1,1
2,1,2021-04-02 00:00:00,2,2
3,1,2021-04-01 00:00:00,1,3
4,2,2021-04-05 00:00:00,4,1
5,2,2021-04-04 00:00:00,3,2
6,2,2021-04-03 00:00:00,2,3


In [110]:
sql = '''with dt_rank as (
select t.*, 
rank() over(partition by t.user_id order by t.dt desc) as rnk
from revenue t)

select * from dt_rank t
where t.rnk = 1

'''

In [111]:
select(sql)

,user_id,dt,revenue,rnk
0,1,2021-04-03 00:00:00,3,1
1,1,2021-04-03 00:00:00,1,1
2,2,2021-04-05 00:00:00,4,1


In [116]:
sql = '''
select t.*, 
row_number() over(partition by t.user_id order by t.dt desc) as rnb
from revenue t
'''

row_number показывает номер строки в определенной партиции 

In [117]:
select(sql)

,user_id,dt,revenue,rnb
0,1,2021-04-03 00:00:00,3,1
1,1,2021-04-03 00:00:00,1,2
2,1,2021-04-02 00:00:00,2,3
3,1,2021-04-01 00:00:00,1,4
4,2,2021-04-05 00:00:00,4,1
5,2,2021-04-04 00:00:00,3,2
6,2,2021-04-03 00:00:00,2,3


In [122]:
sql = '''with dt_rnm as (
select t.*, 
row_number() over(partition by t.user_id order by t.dt desc) as rnb
from revenue t)

select * from dt_rnm t
where t.rnb = 1

'''

In [123]:
select(sql)

,user_id,dt,revenue,rnb
0,1,2021-04-03 00:00:00,3,1
1,2,2021-04-05 00:00:00,4,1


In [18]:
t = pd.DataFrame({'dep':['a','a','a','a','a',
                         'b','b','b','b','b'],
                  'emp':['aa','bb','cc','dd','ee',
                         'aa','bb','cc','dd','ee'],
                  'sal':[5,5,3,2,1,
                         5,4,3,2,1]})

t.to_sql('salary', con, index=False, if_exists='replace')

deportment
employer
salary 
\n
top salaries

In [19]:
t

,dep,emp,sal
0,a,aa,5
1,a,bb,5
2,a,cc,3
3,a,dd,2
4,a,ee,1
5,b,aa,5
6,b,bb,4
7,b,cc,3
8,b,dd,2
9,b,ee,1


In [27]:
sql = ''' with salary_rnk as (
select t.*,
dense_rank() over(partition by t.dep order by t.sal desc) as rnk
from salary t )

select * from salary_rnk t

where t.rnk <= 3
'''

In [28]:
select(sql)

,dep,emp,sal,rnk
0,a,aa,5,1
1,a,bb,5,1
2,a,cc,3,2
3,a,dd,2,3
4,b,aa,5,1
5,b,bb,4,2
6,b,cc,3,3


In [43]:
user1 = pd.DataFrame({'user_id':[1,1,1,1,1],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-01 9:31','2021-04-01 10:31','2021-04-01 11:31','2021-04-03 10:31'],format='%Y-%m-%d %H:%M')})

t.to_sql('user1', con, index=False, if_exists='replace')

In [44]:
user1

,user_id,dt
0,1,2021-04-01 07:31:00
1,1,2021-04-01 09:31:00
2,1,2021-04-01 10:31:00
3,1,2021-04-01 11:31:00
4,1,2021-04-03 10:31:00


In [45]:
user2 = pd.DataFrame({'user_id':[2,2,2,2],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-01 7:31',
                                       '2021-04-01 11:31','2021-04-03 10:31'],
                                      format='%Y-%m-%d %H:%M')})

user2.to_sql('user2', con, index=False, if_exists='replace')

In [46]:
user2

,user_id,dt
0,2,2021-04-01 07:31:00
1,2,2021-04-01 07:31:00
2,2,2021-04-01 11:31:00
3,2,2021-04-03 10:31:00


In [47]:
user3 = pd.DataFrame({'user_id':[3,3,3],
                  'dt':pd.to_datetime(['2021-04-01 07:31','2021-04-03 7:31',
                                       '2021-04-02 11:31'],
                                      format='%Y-%m-%d %H:%M')})

user3.to_sql('user3', con, index=False, if_exists='replace')

In [48]:
user3

,user_id,dt
0,3,2021-04-01 07:31:00
1,3,2021-04-03 07:31:00
2,3,2021-04-02 11:31:00


In [49]:
t = pd.concat([user1,user2,user3])

объединяем датафреймы в один

In [50]:
t

,user_id,dt
0,1,2021-04-01 07:31:00
1,1,2021-04-01 09:31:00
2,1,2021-04-01 10:31:00
3,1,2021-04-01 11:31:00
4,1,2021-04-03 10:31:00
0,2,2021-04-01 07:31:00
1,2,2021-04-01 07:31:00
2,2,2021-04-01 11:31:00
3,2,2021-04-03 10:31:00
0,3,2021-04-01 07:31:00


In [51]:
t.to_sql('client_log',con, index=False, if_exists='replace')

In [104]:
sql = '''
with new_session as (
select t.*,
lag(t.dt) over(partition by t.user_id order by t.dt) as prev_dt,

((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) as dt_diff,

case when ((julianday(t.dt) - julianday(lag(t.dt) over (partition by t.user_id order by t.dt))) * 24 * 60 * 60) >= 3600
then 1 else 0 end as new_session

from client_log t),

client_session as (
select t.*,
sum(t.new_session) over (partition by t.user_id order by t.dt) as session_id

from new_session t),

client_session_agg as(

select t.user_id, t.session_id, count() as action_cnt from client_session t

group by t.user_id, t.session_id)

select count(*) from client_session_agg t
'''

lag позволяет вывести предыдущее значение в партиции

In [105]:
select(sql)

,count(*)
0,9


In [106]:
t = pd.DataFrame({'user_id':[1,1,1,1,1,1,
                             2,2,2,2,2],
                  'dt':[1,2,3,4,5,6,
                        1,2,3,4,5],
                  'revenue':[1,2,3,4,5,6,
                             3,4,5,6,7]})
t.to_sql('revenue',con,index=False,if_exists='replace')

In [109]:
t

,user_id,dt,revenue
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5
5,1,6,6
6,2,1,3
7,2,2,4
8,2,3,5
9,2,4,6


In [114]:
sql = '''
select t.*,
avg(t.revenue) over(partition by t.user_id order by t.dt

rows between 2 preceding and current row
) as moving_avg


from revenue t
'''

Скользящее среднее. Берем среднее арифметическое у актуальной строки с двух предыдущих

In [115]:
select(sql)

,user_id,dt,revenue,moving_avg
0,1,1,1,1.0
1,1,2,2,1.5
2,1,3,3,2.0
3,1,4,4,3.0
4,1,5,5,4.0
5,1,6,6,5.0
6,2,1,3,3.0
7,2,2,4,3.5
8,2,3,5,4.0
9,2,4,6,5.0
